## Parte 1: Buscando texto de uma URL

In [44]:
!pip install requests beautifulsoup4 openai langchain_openai

In [27]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    # Extrair o texto da url
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()
        texto = soup.get_text(separator= ' ')
        # Limpar texto
        linhas = (line.strip() for line in texto.splitlines())
        parts = (phrase.strip() for line in linhas for phrase in line.split("  "))
        texto_limpo = '\n'.join(part for part in parts if part)
        return texto_limpo
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")
        return None

    # Pegar o texto extraído do html da página e gerar arquivo temporário
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    return text

extract_text_from_url('https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo')

"Azure Open AI in VNet - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by\nSearch\nAlgolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nBoost\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nKenichiro Nakamura\nPosted on\nOct 12, 2023\nAzure Open AI in VNet\n# azure\n# openai\n# security\nGPT models are hosted in multiple service vendor at the moment, and Microsoft Azure is one of them.\nEven though the models themselves are the same, there are many differences including:\ncost\nfunctionalities\ntype of models and versions\ngeo location\nsecurity\nsupport\netc.\nOne of the most important aspects when we use it in an Enterprise Environment is, of course, security.\nBy using Azure network security features with Azure Open AI, customers can consume the Open AI service from and within the

## Parte 2: Usando o langchain

In [46]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

In [50]:
!pip install --upgrade langchain_openai

In [76]:
client = AzureChatOpenAI(
    azure_endpoint= "https://mchll-m3ywrwfj-eastus2.openai.azure.com/", #"https://oai-dio-bootcamp-dev-eastus001.openai.azure.com/",
    api_key= "Cli5oticqyS4vhUvMoFnqbnuyN94QB7Zrcp77tFSWYhoCf6im8eBJQQJ99AKACHYHv6XJ3w3AAAAACOGDDEn",
    api_version= "2024-02-15-preview",
    deployment_name= "gpt-4o-mini",
    max_retries=0
)


def translate_article(text, lang):
    messages = [
        ("system" , "Você atua como tradutor de textos"),
        ("user" , f"Traduza o {text} para o idioma {lang} e responda em markdown")
    ]

    response = client.invoke(messages)
    print(response.content)
    return response.content


translate_article("GPT models are hosted in multiple service.", "portugues") # poderia ser "pt-br" também


Os modelos GPT estão hospedados em vários serviços.


'Os modelos GPT estão hospedados em vários serviços.'

### Outra forma de fazer:

Como a camada gratuita pode ter limitações na quantidade de caracteres é possível limitar a quantidade usando o parâmetro `chunk_size`:

In [78]:
def translate_article(text, lang, chunk_size=500):
    """Translates text into the specified language, splitting it into chunks if necessary.

    Args:
        text: The text to translate.
        lang: The target language.
        chunk_size: The maximum number of characters per chunk.

    Returns:
        The translated text.
    """
    if len(text) > chunk_size:
        chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
        translated_chunks = [translate_chunk(chunk, lang) for chunk in chunks]
        return ''.join(translated_chunks)
    else:
        return translate_chunk(text, lang)


def translate_chunk(text, lang):
    """Translates a single chunk of text into the specified language."""
    messages = [
        ("system" , "Você atua como tradutor de textos"),
        ("user" , f"Traduza o {text} para o idioma {lang} e responda em markdown")
    ]
    response = client.invoke(messages)
    return response.content


translate_article("GPT models are hosted in multiple service.", "portugues") # poderia ser "pt-br" também

'Os modelos GPT estão hospedados em vários serviços.'

## Tradutor de URL

In [82]:
# O trecho de código está escrito corretamente e é funcional, porém, como utilizei a camada gratuita o limite de caracteres foi excedido.
"""
url= "https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo"
text= extract_text_from_url(url)
article= translate_article(text, "pt-br")

print(article)
"""

'\nurl= "https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo"\ntext= extract_text_from_url(url)\narticle= translate_article(text, "pt-br")\n\nprint(article)\n'